4-8 End of Assessment Tasks

01 import datasets and libraries

In [1]:
#01.1 import libraries
import pandas as pd
import numpy as np
import os

In [3]:
#01.2 create path to dataframe files
path=r'D:\Joan\CareerFoundry\Assessment 4\2024-08-Instacart Basket Analysis\02 Data\Prepared Data'

In [5]:
#01.3 import dataset
df_ords_prods_merge=pd.read_pickle(os.path.join(path, 'ords_prods_merge_47.pkl'))

In [7]:
#01.4 check shape of df_ords_prods_merge
df_ords_prods_merge.shape

(32435059, 19)

02 Aggregated Mean for Entire Dataframe

In [9]:
#02.1 see dataframe columns
df_ords_prods_merge.columns

Index(['order_id', 'user_id', 'order_number', 'orders_day_of_week',
       'order_hour_of_day', 'days_since_prior_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_range_loc', 'busiest_day',
       'busiest_days', 'busiest_period_of_day', 'max_order', 'loyalty_flag'],
      dtype='object')

In [11]:
#02.2 Aggregated mean of order_number grouped by department_id
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1.0,15.457838
2.0,17.277920
3.0,17.170395
4.0,17.811403
5.0,15.215751
6.0,16.439806
7.0,17.225802
8.0,15.340650


02.1 From eyeballing the numbers I don't see a lot of difference between the subset results and the results of the full dataframe. On average most orders include department id 21 in both results. Departments 1 and 5 are the lowest on average in both results. 

03 Create Loyalty Flag - We did this in during the assessment. It is already in my updated dataset.

04 Marketing (prices of products purchased by customer loyalty)

In [20]:
#04.1 Group df_ords_prods_merge by loyalty_flag
df_ords_prods_merge.groupby('loyalty_flag')

In [22]:
#04.2 Aggregate new grouped object
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices' : ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0

05 Marketing Create Spending Flag

In [40]:
#05.1 split data by user_id based on average prices and create new column average_price
df_ords_prods_merge['average_price']=df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\jonba\AppData\Local\Temp\ipykernel_3088\1272708729.py:2: FutureWarning: The provided callable <function mean at 0x0000027AAFDB6FC0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ords_prods_merge['average_price']=df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [42]:
#05.2 check shape of data
df_ords_prods_merge.shape

(32435059, 21)

In [44]:
#05.2 show first five rows
df_ords_prods_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,department_id,prices,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,spending_flag,average_price
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,7.0,9.0,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,6.367797
1,2539329,1,1,2,8,NaN,14084,2,0,Organic Unsweetened Vanilla Almond Milk,...,16.0,12.5,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,6.367797
2,2539329,1,1,2,8,NaN,12427,3,0,Original Beef Jerky,...,19.0,4.4,Low-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,6.367797
3,2539329,1,1,2,8,NaN,26088,4,0,Aged White Cheddar Popcorn,...,19.0,4.7,Low-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,6.367797
4,2539329,1,1,2,8,NaN,26405,5,0,XL Pick-A-Size Paper Towel Rolls,...,17.0,1.0,Low-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,6.367797


In [72]:
#05.3 create parameters for categories for spending_flag
df_ords_prods_merge.loc[df_ords_prods_merge['average_price'] < 10, 'spending_flag'] = 'Low spender'
df_ords_prods_merge.loc[df_ords_prods_merge['average_price'] >= 10, 'spending_flag'] = 'High spender'

In [74]:
#05.4 check frequency of df_ords_prods_merge['spending_flag']
df_ords_prods_merge['spending_flag'].value_counts()

spending_flag
Low spender     31800381
High spender      634678
Name: count, dtype: int64

06 Marketing Order Frequency Flag

In [76]:
#06.1 split data by user_id based on average prices and create new column average_price
df_ords_prods_merge['median_days_since_prior_order']=df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\jonba\AppData\Local\Temp\ipykernel_3088\528451255.py:2: FutureWarning: The provided callable <function median at 0x0000027AAFFB1D00> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_ords_prods_merge['median_days_since_prior_order']=df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [77]:
#06.2 check shape of data
df_ords_prods_merge.shape

(32435059, 23)

In [80]:
#06.3 create parameters for categories for frequency_flag
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['median_days_since_prior_order'] > 10) & (df_ords_prods_merge['median_days_since_prior_order'] <= 20), 'frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_prior_order'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [82]:
#06.4 check frequency for df_ords_prods_merge['frequency_flag']
df_ords_prods_merge['frequency_flag'].value_counts()

frequency_flag
Frequent customer        21578273
Regular customer          7217066
Non-frequent customer     3639720
Name: count, dtype: int64

07 Export Data

In [ ]:
#07.1 export data as a pickle file
df_ords_prods_merge.to_pickle(os.path.join(path, '